# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
file_path = "../Resources/cities.csv"
city_df = pd.read_csv(file_path)
city_df["Humidity"].max()

100

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [12]:
locations = city_df[["Lat","Lng"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# A max temperature lower than 300 degrees but higher than 200
# Wind speed less than 7 mph
# cloudiness less than 70%
# Drop any rows that don't contain all three conditions
narrowed_city_df = city_df.loc[(city_df["Max Temp"]<300)
                               &(city_df["Max Temp"]>200)
                               &(city_df["Wind Speed"]<7)
                               &(city_df["Cloudiness"]<70)
                               ,:]

narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df = narrowed_city_df.reset_index().drop(columns={"index"})
narrowed_city_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ribeira Grande,38.52,-28.70,294.50,86,35,5.25,PT,1601165287
1,Saldanha,-33.01,17.94,279.15,75,30,0.50,ZA,1601164993
2,Rikitea,-23.12,-134.97,296.79,83,0,6.37,PF,1601165232
3,Hermanus,-34.42,19.23,279.82,93,45,3.57,ZA,1601164993
4,Lagoa,39.05,-27.98,294.10,83,1,6.06,PT,1601165233


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
narrowed_city_df["Hotel Name"]=""
hotel_df = narrowed_city_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ribeira Grande,38.52,-28.70,294.50,86,35,5.25,PT,1601165287,
1,Saldanha,-33.01,17.94,279.15,75,30,0.50,ZA,1601164993,
2,Rikitea,-23.12,-134.97,296.79,83,0,6.37,PF,1601165232,
3,Hermanus,-34.42,19.23,279.82,93,45,3.57,ZA,1601164993,
4,Lagoa,39.05,-27.98,294.10,83,1,6.06,PT,1601165233,


In [8]:
params = {'key':g_key,
         'radius':5000,
         'type':'lodging'}

for index,row in narrowed_city_df.iterrows():
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    locations = f'{row["Lat"]},{row["Lng"]}'
    
    params["location"] = locations
    hotel_data = requests.get(url,params=params).json()
    
    try:
        hotel_df.loc[index,['Hotel Name']] = hotel_data["results"][0]["name"]
    except:
        print(f"Hotel in {row['City']} is not found.")
    
hotel_df.head()     

Hotel in Albany is not found.
Hotel in Abu Samrah is not found.
Hotel in Mīrābād is not found.
Hotel in Nālūt is not found.
Hotel in Beloha is not found.
Hotel in Provideniya is not found.
Hotel in Oruzgan is not found.
Hotel in Sobolevo is not found.
Hotel in Uyovu is not found.
Hotel in Novyy Urgal is not found.
Hotel in Koslan is not found.
Hotel in Vostok is not found.
Hotel in Envira is not found.
Hotel in Ust'-Kulom is not found.
Hotel in Shagonar is not found.
Hotel in Defiance is not found.
Hotel in Mago is not found.
Hotel in Saedinenie is not found.
Hotel in Madimba is not found.
Hotel in Baharly is not found.
Hotel in Camacupa is not found.
Hotel in Blagoyevo is not found.
Hotel in Taywarah is not found.
Hotel in Micheweni is not found.
Hotel in Sinegor'ye is not found.
Hotel in Caxito is not found.
Hotel in Wattegama is not found.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Ribeira Grande,38.52,-28.70,294.50,86,35,5.25,PT,1601165287,Quinta da Meia Eira
1,Saldanha,-33.01,17.94,279.15,75,30,0.50,ZA,1601164993,Blue Bay Lodge
2,Rikitea,-23.12,-134.97,296.79,83,0,6.37,PF,1601165232,Pension Maro'i
3,Hermanus,-34.42,19.23,279.82,93,45,3.57,ZA,1601164993,Misty Waves Boutique Hotel
4,Lagoa,39.05,-27.98,294.10,83,1,6.06,PT,1601165233,Casa Das Faias


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations,info_box_content = hotel_info)
fig.add_layer(marker_layer)
# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))